In [1]:
import pandas as pd
import numpy as np
from buah.models import Dataset
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


In [2]:
dataset = pd.DataFrame(list(Dataset.objects.all().values()))


In [12]:
columns = ['id','form_factor', 'aspect_ratio', 'rect',
               'narrow_factor', 'prd', 'plw', 'idm', 'entropy', 'asm', 'contrast',
               'correlation', 'kelas']
df = dataset.loc[:, columns].copy()
df['id'] = range(1,201)


In [ ]:
df.to_excel("data asli.xlsx", index=False) 

In [15]:

persen_uji = 10 / 100
train_df, test_df = train_test_split(
        df, test_size=persen_uji, stratify=df.loc[:, ['kelas']], random_state=0)
train_df = train_df.sort_index()
test_df = test_df.sort_index()


In [ ]:
train_df.to_excel("data latih belum normalisasi.xlsx", index=False)

In [16]:
test_df.to_excel("data uji belum normalisasi.xlsx", index=False)

In [17]:
train_data = train_df.copy()
test_data = test_df.copy()

X_train = train_data.loc[:, ~train_data.columns.isin(['id', 'kelas'])]
Y_train = train_data.loc[:, train_data.columns.isin(['kelas'])]
X_test = test_data.loc[:, ~test_data.columns.isin(['id', 'kelas'])]
Y_test = test_data.loc[:, test_data.columns.isin(['kelas'])]

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled_with_class = np.concatenate(
        (X_train_scaled, Y_train.to_numpy()), axis=1)

X_test_scaled_with_class = np.concatenate(
        (X_test_scaled, Y_test.to_numpy()), axis=1)

In [59]:
df_X_train_scaled_with_class = pd.DataFrame(X_train_scaled_with_class)
df_X_train_scaled_with_class.insert(0, 'id', train_data['id'].to_list())
df_X_train_scaled_with_class.columns = train_data.columns
df_X_train_scaled_with_class['kelas'] = pd.to_numeric(df_X_train_scaled_with_class['kelas'], downcast='integer')
df_X_train_scaled_with_class.to_excel("data latih normalisasi + kelas.xlsx", index=False)

In [60]:
df_X_test_scaled_with_class = pd.DataFrame(X_test_scaled_with_class)
df_X_test_scaled_with_class.insert(0, 'id', test_data['id'].to_list())
df_X_test_scaled_with_class.columns = test_data.columns
df_X_test_scaled_with_class['kelas'] = pd.to_numeric(df_X_test_scaled_with_class['kelas'], downcast='integer')
df_X_test_scaled_with_class.to_excel("data uji normalisasi + kelas.xlsx", index=False)

In [54]:
df['Column header']

KeyError: 'Column header'

In [19]:
np.savetxt('output_training_80%.dat', X_train_scaled_with_class, fmt='%.5f')
